In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import pandas_profiling
from pandas_profiling  import ProfileReport
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("../../data/raw/meteors.csv")
df

dc = (
    df.rename(columns={"name":"Name","nametype":"Classification","recclass": "Class","mass": "Mass", "fall": "FallType"})
    .sort_values("year", ascending=True)
    .dropna()
    .drop(['reclat', 'reclong'], axis=1)
         )
dc

,Name,id,Classification,Class,Mass,FallType,year,GeoLocation
38301,Wietrzno-Bobrka,24259,Valid,Iron,376.0,Found,601.0,"(49.416670, 21.700000)"
703,Nogata,16988,Valid,L6,472.0,Fell,860.0,"(33.725000, 130.750000)"
278,Elbogen,7823,Valid,"Iron, IID",107000.0,Fell,1399.0,"(50.183330, 12.733330)"
852,Rivolta de Bassi,22614,Valid,Stone-uncl,103.3,Fell,1490.0,"(45.483330, 9.516670)"
283,Ensisheim,10039,Valid,LL6,127000.0,Fell,1491.0,"(47.866670, 7.350000)"
...,...,...,...,...,...,...,...,...
30774,Northwest Africa 7858,57423,Valid,H4,459.0,Found,2013.0,"(0.000000, 0.000000)"
30776,Northwest Africa 7861,57425,Valid,L5,611.0,Found,2013.0,"(0.000000, 0.000000)"
30777,Northwest Africa 7862,57426,Valid,L4/5,317.0,Found,2013.0,"(0.000000, 0.000000)"
30759,Northwest Africa 7812,57258,Valid,Angrite,46.2,Found,2013.0,"(0.000000, 0.000000)"


### Has there been an increase in the mass of meteorites that have fallen over the past 200 years?

- manipulating the dataframe to reflect the data we want more effectively


In [4]:
#create a more reasonable timeframe (of years)
dfy = (dc[dc.year>1800])

#drop the predicted meteorite impact
dff = dfy.drop([30679]) 

#include only significantly sized meteors
dfm = (dff[dff.Mass>1000000]) 

#remove any invalid geolocations
dfgl = dfm[~dfm.GeoLocation.str.contains("0.000000")] 
dfgl

#begin numbering at 1
df = (dfgl
    .reset_index()
    .drop(columns=['index'])
     )
df 

,Name,id,Classification,Class,Mass,FallType,year,GeoLocation
0,Charcas,5326,Valid,"Iron, IIIAB",1400000.0,Found,1804.0,"(23.083330, -101.016670)"
1,Bitburg,5062,Valid,"Iron, IAB complex",1500000.0,Found,1805.0,"(49.966670, 6.533330)"
2,Cape York,5262,Valid,"Iron, IIIAB",58200000.0,Found,1818.0,"(76.133330, -64.933330)"
3,Gibeon,10912,Valid,"Iron, IVA",26000000.0,Found,1836.0,"(-25.500000, 18.000000)"
4,Coahuila,5387,Valid,"Iron, IIAB",2100000.0,Found,1837.0,"(28.700000, -102.733330)"
5,Chupaderos,5363,Valid,"Iron, IIIAB",24300000.0,Found,1852.0,"(27.000000, -105.100000)"
6,Cranbourne,5463,Valid,"Iron, IAB-MG",8600000.0,Found,1854.0,"(-38.100000, 145.300000)"
7,Vaca Muerta,24142,Valid,Mesosiderite-A1,3828000.0,Found,1861.0,"(-25.750000, -70.500000)"
8,Wabar,24194,Valid,"Iron, IIIAB",2550000.0,Found,1863.0,"(21.499720, 50.472220)"
9,Bacubirito,4919,Valid,"Iron, ungrouped",22000000.0,Found,1863.0,"(26.200000, -107.833330)"
